# kNN Classifier

## Implement the Algorithm

In [1]:
import numpy as np

In [2]:
class KnnClassifier:
    def __init__(self, k = 3, distance_function = None):
        self.k = k
        if distance_function is None:
            self.distance_function = self._euclidean_distance
        else:
            self.distance_function = distance_function
        
    def fit(self, X, y):
        self.X_train = X
        self.y_train = y
    
    def _euclidean_distance(self, x1, x2):
        return np.sqrt(np.sum((x1 - x2) ** 2))

    def _manhattan_distance(self, x1, x2):
        pass

    def _minkowski_distance(self, x1, x2, p=3):
        pass

    def predict(self, X):
        predictions = []
        for index, x in enumerate(X):
            distances = [self.distance_func(x, x_train) for x_train in self.X_train]
            # indices of the k nearest neighbors
            k_indices = np.argsort(distances)[:self.k]
            # labels of the k nearest neighbors
            k_neighbor_labels = self.y_train[k_indices]
            # majority vote
            counts = np.bincount(k_neighbor_labels.astype(int))
            predicted_label = np.argmax(counts)
            predictions.append(predicted_label)  
        return np.array(predictions)

